# Contraceptive Demand Forecasting 

Demand forecasting of contraceptive products in Côte D’Ivore.

### Dataset: 
Monthly data of 11 contraceptive products stock across 156 health service delivery sites in the public sector health system in Côte D’Ivore, from January 2016 until June 2019.

### Goal:
Forecast stock demand for July, August and September of 2019.

#### Author: Bruna Correa

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import dates
import seaborn as sns

# Import forecast libraries
from statsmodels.tsa.arima_model import ARIMA,ARIMAResults 
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tools.eval_measures import rmse
from pmdarima import auto_arima

# Neural networks - LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set sns style
sns.set(style="ticks", color_codes=True)

In [ ]:
# Load and check dataset of contraceptives' stock 
stock_df = pd.read_csv('/data/Train.csv')
print(f'Total number of rows: {stock_df.shape[0]}')
print(f'Total number of columns: {stock_df.shape[1]}')
stock_df.head()

In [ ]:
# Features information
stock_df.info()

In [ ]:
# Subselect features that will be used for EDA and forecasting
stock_df_sel = stock_df[['year',
                         'month',
                         'region',
                         'district',
                         'site_code',
                         'product_code',
                         'stock_distributed']].copy()

* Target variable: 'stock_distributed'

## Exploratory Data Analysis

In [ ]:
# Check for missing values
stock_df_sel.isnull().mean().sort_values()

* No missing values.

In [ ]:
# Summarize stock_distributed variable
stock_df_sel['stock_distributed'].describe()

In [ ]:
# Group by site_code, product_code, year, to check time range completeness:
stock_df_sel.groupby(['site_code', 'product_code', 'year']).nunique()['month'].plot(kind='hist')
plt.xlabel('Months per year')
plt.title('Number of months per year with available data', fontsize=15);

* There are missing months for some combinations of site_code + product_code!
* Peak at 6 months: for 2019 we only have data until June.
* To do: populate missing months from Jan-2016 until Jun-2019 and impute values.

In [ ]:
# Plot distribution of stock_distributed variable
plt.figure(figsize=(10, 5))
sns.distplot(a=stock_df_sel['stock_distributed'])
plt.title('stock_distributed', fontsize=15)
plt.tight_layout()

In [ ]:
# Explore data by region
stock_df_sel['region'].value_counts(normalize=True).plot(kind='bar', figsize=(12, 5))
plt.title('Stock distributed by region', size=18)
plt.xlabel('Region')
plt.ylabel('Proportion')
plt.show()

In [ ]:
# Explore data by district
stock_df_sel['district'].value_counts(normalize=True).plot(kind='bar', figsize=(20, 5))
plt.title('Stock distributed by district', size=14)
plt.xlabel('District')
plt.ylabel('Proportion')
plt.show()

In [ ]:
# Explore product's distribution
stock_df_sel['product_code'].value_counts(normalize=True).plot(kind='bar', figsize=(12, 5))
plt.title('Stock distributed by product', size=14)
plt.xlabel('Product')
plt.ylabel('Proportion')
plt.show()

### Temporal EDA

In [ ]:
# Plot distribution of stock_distributed by year
plt.figure(figsize=(5, 5))
sns.boxplot(y='stock_distributed', 
            x='year',
            data=stock_df_sel,
            saturation=1)
plt.title('Distribution of stock_distributed by year', fontsize=14)
plt.xticks(rotation='vertical')
plt.tight_layout()

In [ ]:
# Plot total stock distributed by year
ax = stock_df_sel['year'].value_counts(normalize=False).sort_index().to_frame().transpose().plot.bar()
ax.set_title('Total stock_distributed by year', fontsize=14)
ax.set_ylabel('Count');

* Distributed stock shows an increase across time from 2016 to 2018
* 2019 data is not complete

In [ ]:
# Plot total stock distributed by month
ax = stock_df_sel['month'].value_counts(normalize=False).sort_index().to_frame().transpose().plot.bar()
ax.set_title('Stock distributed by month',fontsize=14)
ax.set_ylabel('Count');

* For the first 6 months, distributed stock shows an increase across time 
* For the last 6 months, we don't have data from 2019, but the same trend seems to occur

In [ ]:
# Check 2019 data to confirm its month range:
stock_df_sel[stock_df_sel['year']==2019]['month'].value_counts(normalize=False).to_frame().sort_index()

* For 2019, there are data until June.

In [ ]:
# Create a datetime formated index for time series analyses
idx = pd.to_datetime(
    dict(
        year=stock_df_sel['year'], 
        month=stock_df_sel['month'], 
        day=1 # as we don't have information about the day, fill it with ones
    )
)
# Set dataframe index
stock_df_sel.set_index(idx, inplace=True)

In [ ]:
# Plot stock distributed from 2016 to 2019
ax = stock_df_sel.groupby(stock_df_sel.index)['stock_distributed'].sum().plot(figsize=(10,5))
ax.set_title('Stock distributed from Jan-2016 to Jun-2019', fontsize=14)
ax.set_ylabel('Count');

* There is a linear uptrend.
* Trend was disrupted in the middle of 2017.

In [ ]:
# Format dataframe to plot a heatmap
monthly = stock_df_sel.resample('M').sum()
monthly['month'] = monthly.index.month
monthly['year'] = monthly.index.year
pv = monthly.pivot("month", "year", "stock_distributed")

In [ ]:
# Plot stock distributed per year
ax = pv.plot(figsize=(10,5))
ax.set_title('Stock distributed per year', fontsize=14)
ax.set_ylabel('Count');

* In 2017 from May to August there was an overall decline in the stock distributed

In [ ]:
# plot the heatmap
sns.heatmap(pv, annot=True);

* Alternative visualization, similar message:
    * stock_distribution increased from Jan-2016 until Jun-2019
    * there was a decline in 2017 (May - Aug)

## Feature engineering

* For some combinations of site_code/product, there are missing dates.
* Populate missing dates and impute values for stock_distributed.

In [ ]:
# 1) Define a function to populate missing months from Jan-2016 until Jun-2019:

def get_ts_range(input_df, date_start, date_end, date_freq):
    
    # Sort data frame by index:
    input_df.sort_index(inplace=True)
    
    # Create an index with every month in the range from Jan-2016 until Jun-2019:
    idx_date = pd.date_range(start=date_start, end=date_end, freq=date_freq)
    
    # Create a new data frame with complete dates and impute it with zeros:
    ts_df = pd.DataFrame(np.zeros(len(idx_date)), index=idx_date)
    
    # Rename col:
    ts_df.columns = ['imputation_col']
    
    # Concatenate input data frame with complete data frame:
    ts_df_comp = pd.concat([ts_df, input_df], axis=1, sort=False)
        
    # Return data frame with complete range:
    return ts_df_comp

In [ ]:
# 2) Define a function to impute values in created months
    
def impute_date_vals(input_df, cat_feat_list, target_feat):
    
    # 1) Impute missing values of stock_distributed with interpolation:
    input_df[target_feat].interpolate(method='linear', inplace=True)
    
    # 2) Impute missing values from begining of time series with back propagation:
    input_df[target_feat].fillna(inplace=True, method='bfill')

    # 3) Impute missing values from end of time series with forward propagation:
    input_df[target_feat].fillna(inplace=True, method='ffill')
    
    # 4) Impute missing values of categorical features (region, district, site_code, product_code) with mode:
    input_df[cat_feat_list] = input_df[cat_feat_list].fillna(input_df.mode().iloc[0])
     
    # 5) Return complete data frame with desired features
    return input_df[cat_feat_list+[target_feat]]

In [ ]:
# 3) Define function to fill and impute individual time series (site_code & product_code)

def get_complete_df(input_df, date_start, date_end, date_freq, cat_feat_list, target_feat):
    
    # Create output place holder df
    df_complete = pd.DataFrame()
    
    # Loop over data frame and get every 'site_code' & 'product_code' combinations:
    for site, product in input_df.groupby(['site_code','product_code']).indices:
        
        # Create a dataframe for each time series:
        single_ts_df = input_df[(input_df['site_code'] == site) & (input_df['product_code'] == product)]
        
        # Add missing months:
        single_ts_df_complete_dates = get_ts_range(single_ts_df, 
                                                   date_start, 
                                                   date_end, 
                                                   date_freq)
        
        # Impute missing values:
        single_ts_df_complete_dates_vals = impute_date_vals(single_ts_df_complete_dates, 
                                                            cat_feat_list,
                                                            target_feat)
        
        # Concatenate individual data frames togetherL
        df_complete = pd.concat([df_complete, single_ts_df_complete_dates_vals[cat_feat_list+[target_feat]]])
    
    # Return complete data frame
    return df_complete

In [ ]:
# Run functions to get a populated and imputed data frame:
stock_df_sel_comp = get_complete_df(input_df = stock_df_sel, 
                                    date_start = '2016-01-01', 
                                    date_end = '2019-06-01', 
                                    date_freq = 'MS',
                                    cat_feat_list = ['region', 'district', 'site_code', 'product_code'],
                                    target_feat = 'stock_distributed')

In [ ]:
# Df info:
stock_df_sel_comp.info

In [ ]:
# Group data by index to check completeness:
stock_df_sel_comp.groupby(stock_df_sel_comp.index).nunique()

In [ ]:
# Check for missing values:
stock_df_sel_comp.isnull().mean().sort_values()

## Forecasting

### ARIMA - Models - Baseline

In [ ]:
# Check data frame formating:
stock_df_sel_comp.head()

In [ ]:
# Define function to run ARIMA with auto_arima for order selection:

def run_auto_arima(input_df, target_feat, train_start, train_end, pred_months):
   
    # Create a placeholder to save prediction losses:
    error = []

    # Loop over data frame to get predictions for each individual time series:
    for site, product in input_df.groupby(['site_code','product_code']).indices:

        # Create a dataframe for each combination of site_code + product code:
        single_ts = input_df[(input_df['site_code'] == site) & (input_df['product_code'] == product)]

        # Set index to monthly data:
        single_ts.index.freq = 'MS'

        # Split data in train and test (last 3 months):
        train = single_ts.loc[train_start:train_end][target_feat]
        test = single_ts.loc[train_end:][target_feat]

        # Run auto_arima to determine ARIMA orders (p,d,q):
        arima_model = auto_arima(train, 
                                 start_p=0, 
                                 start_d=0,
                                 start_q=0,
                                 max_p=6,
                                 max_d=6,
                                 max_q=6,
                                 seasonal=False,
                                 random_state=42)
        #arima_model.summary()

        # Use model to generate predictions for a given number of months:
        prediction = arima_model.predict(n_periods=pred_months)

        # Create predictions data frame to plot:
        #prediction_df = pd.DataFrame(prediction, index=test.index)
        #oprediction_df.columns = [str('predicted_'+ target_feat)]

        # Plot: train, test and predicted values:
        #plt.figure(figsize=(8,5))
        #plt.plot(train)
        #plt.plot(test)
        #plt.plot(prediction_df)
        #plt.xlabel('Date')
        #plt.ylabel('stock_distributed')
        #plt.title(site + '_' + product + ': stock distributed across time')
        #plt.legend(['Train', 'Test', 'Prediction'])
        #plt.savefig('/plots/plot_'+ site + '_' + product + '.pdf')

        # Compare predictions with test, compute RMSE and append to losses:
        error.append(rmse(test, prediction))
    
    # Return average RMSE for all model predictions:
    return np.mean(error)

In [ ]:
# Run ARIMA and get average loss for all time series predictions:
avg_rmse_arima = run_auto_arima(input_df = stock_df_sel_comp, 
                                target_feat = 'stock_distributed',
                                train_start = '2016-01-01',
                                train_end = '2019-04-01',
                                pred_months = 3) # predictions for 3 months

In [ ]:
# Create a dictionary to store losses:
loss_dict = {'rmse_arima': avg_rmse_arima}
loss_dict

In [ ]:
loss_dict = {'rmse_arima': 9.06582051174892}

In [ ]:
# Since trend was interrupted in 2017, try to do predictions starting from 2017-August
avg_rmse_arima_2017 = run_auto_arima(input_df = stock_df_sel_comp, 
                                     target_feat = 'stock_distributed',
                                     train_start = '2017-08-01', # change start month for Aug-2017
                                     train_end = '2019-04-01',
                                     pred_months = 3) # predictions for 3 months

In [ ]:
# Add to losses dict:
loss_dict['rmse_arima_2017'] = avg_rmse_arima_2017
pd.DataFrame(loss_dict, index=[0])

### Neural networks - LSTM

In [ ]:
# Create a subdata frame with one time series
# Subselect one combination of district and product code to test; then loop over all combinations
site = 'C1086'
product = 'AS27133'

#for site, product in stock_df_sel_comp.groupby(['site_code','product_code']).indices:

# Create a dataframe for each combination of site_code + product code
single_ts = stock_df_sel_comp[(stock_df_sel_comp['site_code'] == site) & (stock_df_sel_comp['product_code'] == product)]

# Set index to monthly data
single_ts.index.freq = 'MS'


In [ ]:
# Train / test split

train = single_ts[['stock_distributed']].iloc[:39]
test = single_ts[['stock_distributed']].iloc[39:]

# Scale data
scaler = MinMaxScaler()

# Fit on the train data only, otherwise we using information from the test set
scaler.fit(train)

# Scale train and test
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

# Time series generator
# Define time series generator
# Let's redefine to get 12 months back and then predict the next month out
n_input = 38
n_features = 1
generator = TimeseriesGenerator(scaled_train, 
                                scaled_train, 
                                length=n_input, 
                                batch_size=1)

In [ ]:
# Define model
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# Fit model
model.fit_generator(generator, epochs=15)

In [ ]:
# Plot loss per epoch
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)), loss_per_epoch)

In [ ]:
# Test predictions
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # Get prediction for 1 month ahead:
    current_pred = model.predict(current_batch)[0]
    
    # Store prediction:
    test_predictions.append(current_pred) 
    
    # Update batch: include prediction and drop first value:
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
# Re-scale to original scale
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
test['predictions'] = true_predictions
test

In [ ]:
# Get RMSE
rmse(test['stock_distributed'], test['predictions'])

In [ ]:
# Plot
test.plot(figsize=(12,8))

In [ ]:
# Save model
model.save('rnn_model.h5')

In [ ]:
# Load model
my_model = load_model('rnn_model.h5')

In [ ]:
##### Here

In [ ]:
# Define function to scale data

def data_scaler(input_train, input_test):
    # Scale data
    scaler = MinMaxScaler()

    # Fit on the train data only, otherwise we using information from the test set
    scaler.fit(input_train)

    # Scale train and test
    scaled_train = scaler.transform(input_train)
    scaled_test = scaler.transform(input_test)
    
    return scaled_train, scaled_test

In [ ]:
#aa = stock_df_sel_comp[stock_df_sel_comp['site_code'] == 'C1066']

In [ ]:
# Define model:

n_input = 38
n_features = 1
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Define function to build and get predictions from recurrent neural network with LSTM:

def my_lstm_model(input_df, input_model, n_input, n_features, target_feat, n_epochs):

    # Create a placeholder to save prediction losses:
    error = []

    # Loop over data frame to get predictions for each individual time series:
    for site, product in input_df.groupby(['site_code','product_code']).indices:
    
        # Create a data frame per each time series:
        single_ts = input_df[(input_df['site_code'] == site) & (input_df['product_code'] == product)]

        # Set index to monthly data:
        single_ts.index.freq = 'MS'
    
        # Train / test split:
        train = single_ts[[target_feat]].iloc[:39]
        test = single_ts[[target_feat]].iloc[39:]
    
        # Scale data:
        scaler = MinMaxScaler()

        # Fit on the train data only, otherwise we using information from the test set
        scaler.fit(train)

        # Scale train and test:
        scaled_train = scaler.transform(train)
        scaled_test = scaler.transform(test)
    
        # Define time series generator:
        generator = TimeseriesGenerator(scaled_train, 
                                        scaled_train, 
                                        length=n_input, 
                                        batch_size=1)

        # Fit model:
        input_model.fit_generator(generator, 
                                  epochs=n_epochs,
                                  verbose=0)

        # Plot loss per epoch
        #loss_per_epoch = model.history.history['loss']
        #plt.plot(range(len(loss_per_epoch)), loss_per_epoch)

        # Test predictions:
        test_predictions = []

        first_eval_batch = scaled_train[-n_input:]
        current_batch = first_eval_batch.reshape((1, n_input, n_features))

        # Loop to get predictions:
        for i in range(len(test)):
    
            # Get prediction for 1 month ahead:
            current_pred = model.predict(current_batch)[0]
    
            # Store prediction:
            test_predictions.append(current_pred) 
    
            # Update batch: include prediction and drop first value:
            current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
        
        # Re-scale predictions to match original scale:
        true_predictions = scaler.inverse_transform(test_predictions)
    
        # Add predictions to test df to calculate loss:
        test['predictions'] = true_predictions
    
        # Plot test vs. predictions:
        #test.plot(figsize=(5,5))
    
        # Compute RMSE:
        error.append(rmse(test[target_feat], test['predictions']))
        
        # Save model:
        #model.save(product + '_' + site + '_rnn_model.h5')
    
    # Return average RMSE for all model predictions:
    return np.mean(error)

In [ ]:
# Run lstm model
avg_rmse_lstm = my_lstm_model(input_df = stock_df_sel_comp, 
                              input_model = model,
                              n_input = n_input, 
                              n_features = n_features, 
                              target_feat = 'stock_distributed',
                              n_epochs = 20)

# Add to losses dict:
loss_dict['rmse_rnn_lstm'] = avg_rmse_lstm
pd.DataFrame(loss_dict, index=[0])

### Prophet

In [ ]:
# Install Prophet
#! conda create --name myenv --yes
#! conda activate myenv --yes
#! conda install -c conda-forge fbprophet --yes

In [ ]:
# Import Prophet
from fbprophet import Prophet

# Format data
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'])

# Train / test split
train = df.iloc[:576]
test = df.iloc[576:]

# Create and fit the model 
m = Prophet()
m.fit(train)

# Create a future placeholder
future = m.make_future_dataframe(periods=3,freq='MS')

# Predict and fill future
forecast = m.predict(future)

# Plot
ax = forecast.plot(x='ds',y='yhat',label='Predictions',legend=True,figsize=(12,8))
test.plot(x='ds',y='y',label='True Miles',legend=True,ax=ax,xlim=('2019-04-01','2019-06-01'))

# Compute rmse
predictions = forecast.iloc[-3:]['yhat']
rmse(predictions,test['y'])